In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import functions as pfunc
from pyspark.sql import SQLContext
from pyspark.sql import Window, types

sc = pyspark.SparkContext(appName="phishingURL-AUTOML-BestModel-Discovery")
sqlContext = SQLContext(sc)

import h2o
from h2o.automl import H2OAutoML
import subprocess
subprocess.run('unset http_proxy', shell=True)
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,3 hours 4 mins
H2O cluster timezone:,UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.1
H2O cluster version age:,17 days
H2O cluster name:,H2O_from_python_siemanalyst_rsmfzl
H2O cluster total nodes:,1
H2O cluster free memory:,4.360 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"locked, healthy"


In [ ]:
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.sql.functions import *


## ynverified=1/0 as phishing
## To be discovered with ML/DL/CNN
def func_url_verified(var1):
    if ( var1.lower()=='yes'):
        return 1
    elif (var1.lower()=='no'):
        return 0
    else :
        return 1
func_url_verified_udf = udf(func_url_verified, IntegerType())    

## URL size function1.
def func_url_lengh(var1):
    return len(var1)
func_url_lengh_udf = udf(func_url_lengh, IntegerType())
## URL isbig=1/0 function2.
def func_url_big(var1):
    if (len(var1) > 80):
        return 1
    else: 
        return 0
func_url_big_udf = udf(func_url_big, IntegerType())

## URL number of '@' chars function3.
def func_url_count_at(var1):
    return var1.count('@')
func_url_count_at_udf = udf(func_url_count_at, IntegerType())

## URL punctuation count function4.
def func_url_count_dot(var1):
    return var1.count('.')
func_url_count_dot_udf = udf(func_url_count_dot, IntegerType())

## URL isipyn=1/0 function5.
## Search if matches pattern : "http://xx.xx.xx.xx/anythingnonspace7662u.wkjw3932"
def func_url_is_ip(var1):
    cleanvar=var1.strip('http://')
    m = re.search('(([2][5][0-5]\.)|([2][0-4][0-9]\.)|([0-1]?[0-9]?[0-9]\.)){3}(([2][5][0-5])|([2][0-4][0-9])|([0-1]?[0-9]?[0-9]))([/]+)([\S]+)',cleanvar)
    if m:
        return 1
    else:
        return 0
func_url_is_ip_udf = udf(func_url_is_ip, IntegerType())    

### Every URL with More than 80 chars, will be massive .

## Get all URL Commom Crawl for Top Level Domains : *.com *.net *.org *.co.uk *.ru 
ccrawl_traindf = sqlContext.read.json("hdfs:///user/siemanalyst/data/raw/commoncrawl/dt=20181112/clean-CC-MAIN-2018-43-index-.*.json")

## Assume URLS as not phishing ynverified=0
ccrawl_trainds = ccrawl_traindf.withColumn('ynverified',lit(0))\
.withColumn('url_length',func_url_lengh_udf(ccrawl_traindf['url']))\
.withColumn('massiveurl',func_url_big_udf(ccrawl_traindf['url']))\
.withColumn('count_at',func_url_count_at_udf(ccrawl_traindf['url']))\
.withColumn('count_dot',func_url_count_dot_udf(ccrawl_traindf['url']))\
.withColumn('url_is_ip',func_url_is_ip_udf(ccrawl_traindf['url']))\
.drop('charset','digest','filename','verification_time','languages','length','mime','mime-detected','offset','status')
ccrawl_trainds.printSchema()

ccrawl_train_pd=ccrawl_trainds.toPandas()  
ccrawl_test_pd=ccrawl_trainds.toPandas().head(8000)

ccrawl_train_hf=h2o.H2OFrame(ccrawl_train_pd)
ccrawl_test_hf=h2o.H2OFrame(ccrawl_test_pd)

### From Phishtank Import Malware urls
#phishtank_trainpd=pd.read_csv("/home/siemanalyst/notebooks/siem/verified_online.csv")
#phishtank_trainds=phishtank_trainpd.drop(columns=['phish_id','phish_detail_url','submission_time','verification_time','online','target'])
#phishtank_trainds['url_length'] = phishtank_trainds['url'].map(str).apply(len)
#phishtank_trainds['massiveurl'] = np.where(phishtank_trainds['url_length'] > 80 , 1, 0)
#phishtank_trainds.head(6)
#phishtank_train=h2o.H2OFrame(phishtank_trainds)
#
#phishtank_testpd=pd.read_csv("/home/siemanalyst/notebooks/siem/verified_online.csv")
#phishtank_testds=phishtank_testpd.drop(columns=['phish_id','phish_detail_url','submission_time','verification_time','online','target'])
#phishtank_testds['url_length'] = phishtank_testds['url'].map(str).apply(len)
#phishtank_testds['massiveurl'] = np.where(phishtank_testds['url_length'] > 80 , 1, 0)
#phishtank_testds.head(6)
#phishtank_test=h2o.H2OFrame(phishtank_testds.head(1000))
#
phishtank_traindf= sqlContext.read.json("hdfs:///user/siemanalyst/data/raw/phishtank/dt=20181112/verified_online.json")
phishtank_traindf.printSchema()

phishtank_trainds=phishtank_traindf\
.withColumn('ynverified',func_url_verified_udf(phishtank_traindf['verified']))\
.withColumn('url_length',func_url_lengh_udf(phishtank_traindf['url']))\
.withColumn('massiveurl',func_url_big_udf(phishtank_traindf['url']))\
.withColumn('count_at',func_url_count_at_udf(phishtank_traindf['url']))\
.withColumn('count_dot',func_url_count_dot_udf(phishtank_traindf['url']))\
.withColumn('url_is_ip',func_url_is_ip_udf(phishtank_traindf['url']))\
.drop('details','online','phish_detail_url','phish_id','submission_time','target','verification_time','verified')
phishtank_trainds.printSchema()

phishtank_train_pd=phishtank_trainds.toPandas()  
phishtank_test_pd=phishtank_trainds.toPandas().head(8000)

phishtank_train=h2o.H2OFrame(phishtank_train_pd)
phishtank_test=h2o.H2OFrame(phishtank_test_pd)


# RBIND "Merge" all of df2 into df1 with rbind .
# function merge() doesn't work if both H2O/dataframes have same variables


###### TRAINING PROCESS ############
#
train = ccrawl_train_hf.rbind(phishtank_train)
test = ccrawl_test_hf.rbind(phishtank_test)

# Identify predictors and response
x = train.columns

# test Learning is it a bit URL TRUE/FALSE
#y = 'massiveurl'

# Casual is it verified phishing from url characteristics TRUE/FALSE
y= 'ynverified'
x.remove(y)

# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML for 40 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=40, seed=1999)
aml.train(x=x, y=y, training_frame=train)


root
 |-- url: string (nullable = true)
 |-- ynverified: integer (nullable = false)
 |-- url_length: integer (nullable = true)
 |-- massiveurl: integer (nullable = true)
 |-- count_at: integer (nullable = true)
 |-- count_dot: integer (nullable = true)
 |-- url_is_ip: integer (nullable = true)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
root
 |-- details: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- announcing_network: string (nullable = true)
 |    |    |-- cidr_block: string (nullable = true)
 |    |    |-- country: string (nullable = true)
 |    |    |-- detail_time: string (nullable = true)
 |    |    |-- ip_address: string (nullable = true)
 |    |    |-- rir: string (nullable = true)
 |-- online: string (nullable = true)
 |-- phish_detail_url: string (nullable = true)
 |-- phish_id: string (nullable = true)
 |-- submissio

In [3]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20181113_131948,0.733923,0.525557,0.339177,0.418926,0.175499
XGBoost_grid_1_AutoML_20181113_131948_model_7,0.733744,0.524633,0.340373,0.41869,0.175301
StackedEnsemble_BestOfFamily_AutoML_20181113_131948,0.733733,0.525785,0.339158,0.419001,0.175562
GBM_grid_1_AutoML_20181113_131948_model_6,0.733416,0.524353,0.338689,0.418617,0.17524
GBM_4_AutoML_20181113_131948,0.73329,0.524395,0.339871,0.418708,0.175316
GBM_grid_1_AutoML_20181113_131948_model_3,0.732098,0.525317,0.34064,0.419061,0.175612
XGBoost_grid_1_AutoML_20181113_131948_model_8,0.73201,0.525537,0.339478,0.419267,0.175785
XGBoost_3_AutoML_20181113_131948,0.731296,0.52753,0.342145,0.419831,0.176258
DRF_1_AutoML_20181113_131948,0.73118,0.558818,0.340471,0.419833,0.17626
XGBoost_grid_1_AutoML_20181113_131948_model_9,0.73106,0.525983,0.338427,0.419508,0.175987


In [7]:
# The leader model is stored here
aml.leader

# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

preds = aml.predict(test)

# or:
#preds = aml.leader.predict(test)


stackedensemble prediction progress: |████████████████████████████████████| 100%


In [8]:
print(preds)
print(test)

predict,p0,p1
1,0.50136,0.49864
1,0.50136,0.49864
1,0.50136,0.49864
1,0.50136,0.49864
1,0.450131,0.549869
1,0.718512,0.281488
1,0.718512,0.281488
1,0.718512,0.281488
0,0.829376,0.170624
1,0.657014,0.342986


url,ynverified,url_length,massiveurl,count_at,count_dot,url_is_ip
http://www2.co.uk/musings/2009/11/,0,34,0,0,2,0
http://www2.co.uk/musings/2009/12/,0,34,0,0,2,0
http://www2.co.uk/musings/2010/05/,0,34,0,0,2,0
http://www2.co.uk/musings/2010/08/,0,34,0,0,2,0
http://www2.co.uk/musings/author/admin/,0,39,0,0,2,0
http://www2.co.uk/musings/category/facebook-privacy/,0,52,0,0,2,0
http://www2.co.uk/musings/category/internet-privacy/,0,52,0,0,2,0
http://www2.co.uk/musings/category/internet-secrecy/,0,52,0,0,2,0
http://www2.co.uk/musings/category/uncategorized/,0,49,0,0,2,0
http://www2.co.uk/musings/facebooks-privacy-policy-mandatory/,0,61,0,0,2,0
